<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>A2: Kaggle Ranking - SF - TEAM_1</h2>
<h4>Business Challenge #2 - BCH-7812 - FMBAN1</h4>
<br>
Professor - Chase Kusterer<br>
Hult International Business School<br>
<h4>Team Members</h4>
<br>
Dennis Otieno
<br>
Rhea Kapoor
<br>
Ofonimo Ben
<br>
Kevin Kabore
<br>
Htet Aung Kyaw

<br>
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

In [1]:
# importing critical libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas                      as pd             # data science essentials
import matplotlib.pyplot           as plt            # essential graphical output
import seaborn                     as sns            # enhanced graphical output
import statsmodels.formula.api     as smf            # regression modeling
import numpy                       as np             # mathematical essentials
import sklearn.linear_model                          # linear modeling in scikit-learn
from sklearn.model_selection import train_test_split # train/test split
from sklearn.tree import plot_tree                   # tree plots
from sklearn.model_selection import RandomizedSearchCV # hyperparameter tuning
from sklearn.ensemble import GradientBoostingRegressor # gbm
from sklearn.linear_model import LogisticRegression  #logistic regression
from sklearn.ensemble import RandomForestClassifier  #Random Forest
from sklearn.metrics import roc_auc_score            #roc_auc_score scoring
from sklearn.metrics import confusion_matrix         #confusion matrix
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.ensemble import GradientBoostingClassifier #gradient boosting


# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))       

Let's Import and read the datasets from the train and test excel files.

In [2]:
# importing the training dataset
path             = "./"
training_dataset = "train.csv"

# reading in the .csv file with pandas
titanic_train    = pd.read_csv(filepath_or_buffer = path + training_dataset)

# checking basic info about the dataset
titanic_train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [3]:
# importing the training dataset
path             = "./"
testing_dataset  = 'test.csv'

# importing the testing dataset
titanic_test = pd.read_csv(filepath_or_buffer = path + testing_dataset)

# checking basic info about the dataset
titanic_test.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


<h2>Joining The Data and Handling Missing Values</h2>
<br>
<br>
We join the data so that we can get a good picture of the whole dataset.
<br>
<br>
Looking for the columns that have missing values.

In [4]:
#Setting the colums for joining the two datasets
titanic_train['set'] = 'Training'
titanic_test ['set'] = 'Testing'

# concatenating both datasets together for mv and feature engineering
titanic_df = titanic_train.append(other = titanic_test)

# resetting index to avoid problems later in the code
titanic_df.reset_index(drop = False,
                       inplace = True)

C:\Users\domuy\AppData\Local\Temp\ipykernel_170664\1196545245.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  titanic_df = titanic_train.append(other = titanic_test)


In [5]:
titanic_df.isnull().sum()

index              0
PassengerId        0
HomePlanet       288
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Transported     4277
set                0
dtype: int64

Based on the data and columns that have missing values, the following will be done to fill the columns with missing values:
<br>
<br>
1. Home planet will be filled with unknown because it is a string.
<br>
<br>
2. CryoSleep will be design in a binary categorical nature to flag out the missing values.
<br>
<br>
3. Cabin will be filled with UNKNOWN/2000/UNKNOWN beacuse the parts for deck and side are objects but the num is an integer
<br>
<br>
4. Destination will be filled with unknown because it is a string.
<br>
<br>
5. Name will be filled with unknown because it is a string.
<br>
<br>
6. Age will be filled with the mean 28.
<br>
<br>
7. VIP will be design in a binary categorical nature to flag out the missing values.
<br>
<br>
8. The continuous columns will be filled with -999.

In [6]:
#filling the missing value in HomePlane with unknown
titanic_df.HomePlanet.fillna("UKNOWN", inplace= True)
#filling the missing value in name with unknown
titanic_df.Name.fillna("UKNOWN", inplace=True)
#filling the missing value in VRDeck with -999
titanic_df.VRDeck.fillna(-999, inplace= True)
#filling the missing value in Spa with -999
titanic_df.Spa.fillna(-999, inplace= True)
#filling the missing value in ShoppingMall with -999
titanic_df.ShoppingMall.fillna(-999, inplace=True)
#filling the missing value in FoodCourt with -999
titanic_df.FoodCourt.fillna(-999, inplace=True)
#filling the missing value in RoomService with -999
titanic_df.RoomService.fillna(-999, inplace=True)
#filling the missing value in Age with average 28
titanic_df.Age.fillna(28, inplace=True)
#filling the missing value in cabin with unknown/2000/2000
titanic_df.Cabin.fillna(f"UKNOWN/{2000}/UNKNOWN", inplace= True)
#creating new binary column for Cryosleep_true
titanic_df["CryoSleep_TRUE"] = 0
#creating new binary column for Cryosleep_false
titanic_df["CryoSleep_False"] = 0
#creating new binary column for vip_true
titanic_df["VIP_TRUE"] = 0
#creating new binary column for vip_true
titanic_df["VIP_FALSE"] = 0
# creating new columns
titanic_df["Surname"] = titanic_df["Name"].copy()
titanic_df["GroupId"] = titanic_df['PassengerId'].copy()
titanic_df["MemberId"] = titanic_df["PassengerId"].copy()
#for loop to look for all the data considering our condition
for index, values in titanic_df.iterrows():
    #condition for creating a binary column cryosleep true
    if titanic_df.loc[index,"CryoSleep"] == True:
        titanic_df.loc[index,'CryoSleep_TRUE'] = 1
    #condition for creating a binary column cryosleep false    
    if titanic_df.loc[index,"CryoSleep"] == False:
        titanic_df.loc[index,'CryoSleep_False'] = 1
    #condition for creating a binary column vip true    
    if titanic_df.loc[index, "VIP"] == True:
        titanic_df.loc[index, 'VIP_TRUE'] = 1
    #condition for creating a binary column vip true    
    if titanic_df.loc[index, "VIP"] == False:
        titanic_df.loc[index, 'VIP_FALSE'] = 1    

<h2>Feature Engineering</h2>
<br>
<br>
The Home Planet feature is being split into three separate boolean variables for Earth, Europa, and Mars. If a passenger is from Earth, then the Earth variable will be set to 1 and the Mars and Europa variables will be set to 0. If a passenger is from Europa or Mars, then their respective variables will be set to 1 and the Earth variable will be set to 0.
<br>
<br>
The CyroSleep variable is being transformed into a binary variable where it takes the value 1 if the passenger was sleeping during the trip and 0 if the passenger was awake. 
<br>
<br>
The Cabin feature contains multiple pieces of information in the form of deck/num/side. To simplify the feature, the decision was made to store each piece of information in a separate feature. The Deck information is being split into eight different features, each corresponding to a different deck: 'Deck_A', 'Deck_B', 'Deck_C', 'Deck_D', 'Deck_E', 'Deck_F', 'Deck_G', 'Deck_T'. If a passenger was residing in a particular deck, then the corresponding feature for that deck will be set to 1, and all other deck features will be set to 0.
<br>
<br>
The Home_Planet to Destination features (Earth_To_Cancries, Earth_To_TRAPPIST, Earth_To_PSO, Mars_To_TRAPPIST, Mars_To_Cancries, Mars_To_PSO, Europa_To_TRAPPIST, Europa_To_PSO, Europa_To_Cancries ) is a combination of two separate features that show where the passenger comes from (Home_Planet) and where they are going (Destination). The feature is set to 1 if the passenger if traveling from is Home planet to the right destination.
<br>
<br>
For the Destination feature, the decision was made to create a binary column for each destination that will be set to 1 if the passenger is traveling in that direction.
<br>
<br>
We then transformed the VIP column into two variables, VIP_True and VIP_False. VIP_True takes the value 1 if the passenger is a VIP, and 0 otherwise. Similarly, VIP_False takes the value 1 if the passenger is not a VIP, and 0 otherwise.
<br>
<br>
To create the Not_Alone feature that identifies passengers who are traveling in groups, we used the passenger_id column, which contains the number of people within each group. And when that number was above 1 we classify that passanger as Not_Alone 
<br>
<br>
The Total Spending feature is the sum of each spending on luxury amenities: RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck. The Wealth_Person feature is built based on the average Total Spending value. Any passenger whose Total Spending value is above the average is considered wealthy.
<br>
<br>
It was observed that the RoomService, FoodCourt, ShoppingMall, Spa, and VRDeck features contained many 0 values, which may represent the fact that the passenger did not purchase these services. Therefore, new binary features were created, including With_RoomService, With_FoodCourt, With_Spa, With_VRDeck, and With_ShoppingMall, which take the value 1 if the passenger spent any amount on the corresponding service and 0 otherwise.
<br>
<br>
We then decided to divide the age category into four different categories. Children include every passenger below 13 years old, Youth includes every passenger from 13 to 35 years old, Adults include every passenger from 35 to 65 years old, and Elder includes every passenger above 65 years old.
<br>
<br>
The Same_Surname feature is binary and is built using passengers who share a similar last name to group people who might be part of the same family. It is set to 1 when the last name of the passenger matches with one or more passengers.
<br>
<br>
Cabin_headcount gives us the average number of people in each cabin who travel with More_than_one_pax, which is a binary feature that shows when a cabin has more than an average of 2 people in it.

In [7]:
#creating binary for the column transported
titanic_df["Transported_New"] = 0
#creating binary for the Home Planet
titanic_df["Earth"] = 0
titanic_df["Europa"] = 0
titanic_df["Mars"] = 0
#creating binary for the Destination column
titanic_df["TRAPPIST"] = 0
titanic_df["Cancri"] = 0
titanic_df["PSO"] = 0
#creating the new columns from the split column cabin
titanic_df["Deck"] = titanic_df['Cabin'].copy()
titanic_df.Deck.fillna(f"UKNOWN/{2000}/UNKNOWN", inplace= True)
titanic_df["Num"] = titanic_df['Cabin'].copy()
titanic_df.Num.fillna(f"UKNOWN/{2000}/UNKNOWN", inplace= True)
titanic_df["Side"] = titanic_df['Cabin'].copy()
titanic_df.Side.fillna(f"UKNOWN/{2000}/UNKNOWN", inplace= True)
#creating another column total lux spending for all the continuous variables
titanic_df['Total_Lux_Spending'] = titanic_df['RoomService'] + titanic_df['FoodCourt'] + titanic_df['ShoppingMall'] + titanic_df['Spa'] + titanic_df['VRDeck']
#creating binary columns for the continuous variables to cater for the zeros
titanic_df["With_Family"] = 0
titanic_df["With_RoomService"] = 0
titanic_df["With_FoodCourt"] = 0
titanic_df["With_Spa"] = 0
titanic_df["With_VRDeck"] = 0
titanic_df["With_ShoppingMall"] = 0
# creating new columns for the age brackets
titanic_df['Teenagers'] = 0
titanic_df['Youth'] = 0
titanic_df['Adults'] = 0
titanic_df['Seniors'] = 0
titanic_df['Children'] = 0
#creating new columns for passengers from a home planet to specific destination
titanic_df["Earth_To_TRAPPIST"] = 0
titanic_df["Earth_To_Cancrie"] = 0
titanic_df["Earth_To_PSO"] = 0
titanic_df["Europa_To_TRAPPIST"] = 0
titanic_df["Europa_To_Cancrie"] = 0
titanic_df["Europa_To_PSO"] = 0
#creating new columns for passengers from a home planet to specific destination
titanic_df["Mars_To_TRAPPIST"] = 0
titanic_df["Mars_To_Cancrie"] = 0
titanic_df["Mars_To_PSO"] = 0
#passengers who have family 
titanic_df["Not_Alone"] = 0
#creating a column for wealthy person
titanic_df["Wealthy_Person"] = 0
#creating a new column for passengers with the same surname
titanic_df["Same_Surname"] = 0
#passengers per cabin 
titanic_df["Cabin_Headcount"] = 0
#passengers from earth and mars as humans and non_humans
titanic_df["Earth_and_Mars"] = 0
titanic_df["More_than_one_pax"] = 0
#accounting for ever deck in the ship
titanic_df["Deck_A"] = 0
titanic_df["Deck_B"] = 0
titanic_df["Deck_C"] = 0
titanic_df["Deck_D"] = 0
titanic_df["Deck_E"] = 0
#accounting for every deck in the ship
titanic_df["Deck_F"] = 0
titanic_df["Deck_G"] = 0
titanic_df["Deck_T"] = 0
#accounting for every side in the ship
titanic_df["Side_P"] = 0
titanic_df["Side_S"] = 0
for index, values in titanic_df.iterrows():
    #condition for creating binary for the column transported
    if titanic_df.loc[index,"Transported"] == True:
        titanic_df.loc[index,"Transported_New"] = 1
    if titanic_df.loc[index,"Transported"] == False:
        titanic_df.loc[index,"Transported_New"] = 0
    #condition for creating binary for the Home Planet    
    if titanic_df.loc[index,"HomePlanet"] == "Earth":
         titanic_df.loc[index,"Earth"] = 1
    if titanic_df.loc[index,"HomePlanet"] == "Europa":
         titanic_df.loc[index,"Europa"] = 1
    if titanic_df.loc[index,"HomePlanet"] == "Mars":
         titanic_df.loc[index,"Mars"] = 1
    #condition for creating binary for the Destination column        
    if titanic_df.loc[index,"Destination"] == "TRAPPIST-1e":
        titanic_df.loc[index,"TRAPPIST"] = 1
    if titanic_df.loc[index,"Destination"] == "55 Cancri e":
        titanic_df.loc[index,"Cancri"] = 1
    if titanic_df.loc[index,"Destination"] == "PSO J318.5-22":
        titanic_df.loc[index,"PSO"] = 1 
    #creating the new columns from the split column cabin    
    titanic_df.loc[index,"Deck"] = titanic_df.loc[index,"Deck"].split(f"/")[0]
    titanic_df.loc[index,"Num"] = titanic_df.loc[index,"Num"].split(f"/")[1]
    titanic_df.loc[index,"Side"] = titanic_df.loc[index,"Side"].split(f"/")[2] 
    
    #creating binary columns for the continuous variables to cater for the zeros
    if titanic_df.loc[index,"RoomService"] != 0 :
        titanic_df.loc[index, "With_RoomService"] = 1

    if titanic_df.loc[index,"FoodCourt"] != 0 :
        titanic_df.loc[index, "With_FoodCourt"] = 1

    if titanic_df.loc[index,"Spa"] != 0 :
        titanic_df.loc[index, "With_Spa"] = 1
    #creating binary columns for the continuous variables to cater for the zeros
    if titanic_df.loc[index,"VRDeck"] != 0 :
        titanic_df.loc[index, "With_VRDeck"] = 1

    if titanic_df.loc[index,"ShoppingMall"] != 0 :
        titanic_df.loc[index, "With_ShoppingMall"] = 1
     
    # creating new columns for the age brackets
    if titanic_df.loc[index, 'Age'] < 13:
        titanic_df.loc[index, 'Children'] = 1   
    
    elif titanic_df.loc[index, 'Age'] >= 13 and titanic_df.loc[index, 'Age'] < 20:
        titanic_df.loc[index, 'Teenagers'] = 1 

    elif titanic_df.loc[index, 'Age'] >= 20 and titanic_df.loc[index, 'Age'] < 35:
        titanic_df.loc[index, 'Youth'] = 1 
    # creating new columns for the age brackets
    elif titanic_df.loc[index, 'Age'] >= 35 and titanic_df.loc[index, 'Age'] < 60:
        titanic_df.loc[index, 'Adults'] = 1

    elif titanic_df.loc[index, 'Age'] >= 60:
        titanic_df.loc[index, 'Seniors'] = 1
      
    try:
        titanic_df.loc[index,"Name"] = titanic_df.loc[index,"Name"].split(" ")[0]
        titanic_df.loc[index,"Surname"] = titanic_df.loc[index,"Surname"].split(" ")[1]
    except:
        titanic_df.loc[index,"Name"] = titanic_df.loc[index,"Name"]
        titanic_df.loc[index,"Surname"] = titanic_df.loc[index,"Surname"]
    #accounting for ever deck in the ship
    if titanic_df.loc[index,"Deck"] == "A":
        titanic_df.loc[index, "Deck_A"] = 1
    elif titanic_df.loc[index,"Deck"] == "B":
        titanic_df.loc[index, "Deck_B"] = 1
    elif titanic_df.loc[index,"Deck"] == "C":
        titanic_df.loc[index, "Deck_C"] = 1
    #accounting for ever deck in the ship    
    elif titanic_df.loc[index,"Deck"] == "D":
        titanic_df.loc[index, "Deck_D"] = 1
    elif titanic_df.loc[index,"Deck"] == "E":
        titanic_df.loc[index, "Deck_E"] = 1
    elif titanic_df.loc[index,"Deck"] == "F":    
        titanic_df.loc[index, "Deck_F"] = 1
    #accounting for ever deck in the ship    
    elif titanic_df.loc[index,"Deck"] == "G":
        titanic_df.loc[index, "Deck_G"] = 1
    elif titanic_df.loc[index,"Deck"] == "T":
        titanic_df.loc[index, "Deck_T"] = 1
        
    #creating new columns for passengers from a home planet to specific destination  
    if titanic_df.loc[index,"HomePlanet"] == "Earth" and \
        titanic_df.loc[index,"Destination"] == "TRAPPIST-1e":
        titanic_df.loc[index, "Earth_To_TRAPPIST"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Earth" and \
        titanic_df.loc[index,"Destination"] == "55 Cancri e":
        titanic_df.loc[index, "Earth_To_Cancrie"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Earth" and \
        titanic_df.loc[index,"Destination"] == "PSO J318.5-22":
        titanic_df.loc[index, "Earth_To_PSO"] = 1
    #creating new columns for passengers from a home planet to specific destination    
    elif titanic_df.loc[index,"HomePlanet"] == "Europa" and \
        titanic_df.loc[index,"Destination"] == "TRAPPIST-1e":
        titanic_df.loc[index, "Europa_To_TRAPPIST"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Europa" and \
        titanic_df.loc[index,"Destination"] == "55 Cancri e":
        titanic_df.loc[index, "Europa_To_Cancrie"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Europa" and \
        titanic_df.loc[index,"Destination"] == "PSO J318.5-22":
        titanic_df.loc[index, "Europa_To_PSO"] = 1
    #creating new columns for passengers from a home planet to specific destination    
    elif titanic_df.loc[index,"HomePlanet"] == "Mars" and \
        titanic_df.loc[index,"Destination"] == "TRAPPIST-1e":
        titanic_df.loc[index, "Mars_To_TRAPPIST"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Mars" and \
        titanic_df.loc[index,"Destination"] == "55 Cancri e":
        titanic_df.loc[index, "Mars_To_Cancrie"] = 1
    elif titanic_df.loc[index,"HomePlanet"] == "Mars" and \
        titanic_df.loc[index,"Destination"] == "PSO J318.5-22":
        titanic_df.loc[index, "Mars_To_PSO"] = 1
    #accounting for every side in the ship   
    if titanic_df.loc[index,"Side"] == "S":
        titanic_df.loc[index, "Side_S"] = 1
    elif titanic_df.loc[index,"Side"] == "P":
        titanic_df.loc[index, "Side_P"] = 1
    #passengers who have family    
    if titanic_df.loc[index,"MemberId"] == 2:
        titanic_df.loc[index, "Not_Alone"] = 1
    elif titanic_df.loc[index,"MemberId"] == 3:
        titanic_df.loc[index, "Not_Alone"] = 1
    elif titanic_df.loc[index,"MemberId"] == 4:
        titanic_df.loc[index, "Not_Alone"] = 1
    #passengers who have family     
    elif titanic_df.loc[index,"MemberId"] == 5:
        titanic_df.loc[index, "Not_Alone"] = 1
    elif titanic_df.loc[index,"MemberId"] == 6:
        titanic_df.loc[index, "Not_Alone"] = 1
    elif titanic_df.loc[index,"MemberId"] == 7:
        titanic_df.loc[index, "Not_Alone"] = 1
    #passengers who have family     
    elif titanic_df.loc[index,"MemberId"] == 8:
        titanic_df.loc[index, "Not_Alone"] = 1
    #creating a column for wealthy person    
    if titanic_df.loc[index,"Total_Lux_Spending"] >= 689  and \
        titanic_df.loc[index,"CryoSleep"] == False:
        titanic_df.loc[index, "Wealthy_Person"] = 1
            
# count same Surname people
name_counts = titanic_df['Surname'].value_counts().to_dict()
# Create a new column 'Count' and map the name counts to each name
titanic_df['Surname_Count'] = titanic_df['Surname'].map(name_counts)

# count same Surname people
head_counts = titanic_df['Cabin'].value_counts().to_dict()
# Create a new column 'Count' and map the name counts to each name
titanic_df['Cabin_Headcount'] = titanic_df['Cabin'].map(head_counts)

for index, values in titanic_df.iterrows():
    if titanic_df.loc[index,"Surname_Count"] != 1:
        titanic_df.loc[index,'Same_Surname'] = 1
#creating a loop for binary family column
for index, values in titanic_df.iterrows():
    if titanic_df.loc[index,"MemberId"] != 1 and \
        titanic_df.loc[index,"Surname_Count"] != 1:
        titanic_df.loc[index,"With_Family"] = 1

#passengers from earth and mars as humans and non_humans
for index, values in titanic_df.iterrows():
    if titanic_df.loc[index,"Deck_D"] == 1 or \
        titanic_df.loc[index,"Deck_E"] == 1 or \
        titanic_df.loc[index,"Deck_F"] == 1:
        titanic_df.loc[index,"Earth_and_Mars"] = 1
        
    #creating a column with passengers who are not alone
    if titanic_df.loc[index,"Cabin_Headcount"] >= 2:
        titanic_df.loc[index,"More_than_one_pax"] = 1    
    

<h2>CLASSIFICATION MODELING</h2>

In [8]:
#dropping some columns which are not important
titanic_df.drop(columns=["HomePlanet", "CryoSleep", "Cabin", "Destination", "VIP", "Transported", "Deck", "Side", "Name", "Surname" ], axis=1, inplace=True)

In [9]:
# instantiating a correlation matrix
titanic_corr = titanic_df[ titanic_df['set']   == 'Training' ].corr(method = 'pearson').round(decimals = 2)

# transforming correlations to absolute values
titanic_corr.loc[ : , 'Transported_New' ].apply(func = abs).sort_values(ascending = False)

Transported_New       1.00
CryoSleep_TRUE        0.46
CryoSleep_False       0.45
Wealthy_Person        0.40
With_RoomService      0.35
With_Spa              0.34
With_VRDeck           0.33
With_ShoppingMall     0.26
With_FoodCourt        0.23
RoomService           0.23
Spa                   0.22
Total_Lux_Spending    0.20
VRDeck                0.20
Europa                0.18
Earth_To_TRAPPIST     0.17
Earth                 0.17
Earth_and_Mars        0.16
More_than_one_pax     0.15
Deck_B                0.14
Europa_To_Cancrie     0.13
Children              0.13
Cancri                0.11
Deck_C                0.11
Side_P                0.10
Side_S                0.10
Europa_To_TRAPPIST    0.10
Deck_E                0.10
Deck_F                0.09
TRAPPIST              0.09
Age                   0.07
Youth                 0.07
VIP_TRUE              0.04
FoodCourt             0.04
Mars_To_Cancrie       0.03
With_Family           0.03
Same_Surname          0.03
Deck_D                0.03
D

In [10]:
# setting explanatory variable(s) with most correlated x-variable
x_train = titanic_df[["CryoSleep_TRUE",
                      "CryoSleep_False",
                      "Wealthy_Person",
                      "With_RoomService",
                      "With_Spa",
                      "With_VRDeck",
                      "With_ShoppingMall",
                      "RoomService",
                      "Spa",
                      "VRDeck",
                      "Total_Lux_Spending",
                      "Earth_To_TRAPPIST",
                      "Deck_C",
                      "Side_P",
                      "Deck_E",
                      "With_Family",
                      "Mars",
                      "Teenagers",
                      "PSO",
                      "Earth_To_Cancrie",
                      'More_than_one_pax'
                      ]].copy()[ titanic_df['set'] == 'Training' ]

# setting response variable
y_train = titanic_df['Transported_New'][ titanic_df['set']   == 'Training' ]

In [11]:

for val in x_train:
    print(f" {val} + ")

 CryoSleep_TRUE + 
 CryoSleep_False + 
 Wealthy_Person + 
 With_RoomService + 
 With_Spa + 
 With_VRDeck + 
 With_ShoppingMall + 
 RoomService + 
 Spa + 
 VRDeck + 
 Total_Lux_Spending + 
 Earth_To_TRAPPIST + 
 Deck_C + 
 Side_P + 
 Deck_E + 
 With_Family + 
 Mars + 
 Teenagers + 
 PSO + 
 Earth_To_Cancrie + 
 More_than_one_pax + 


In [12]:
# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train,
            random_state = 123,
            test_size    = 0.25,
            stratify     = y_train)

In [13]:
# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train,
            random_state = 123,
            test_size    = 0.25,
            stratify     = y_train)

#Gradient Boosting
# specifying a model name
model_name = 'Unpruned Gradient Boosting Classifier'
ugbc = GradientBoostingClassifier(loss          = 'exponential',
                                    min_samples_split= 100,
                                    min_samples_leaf= 103,
                                    criterion     = 'squared_error',
                                    n_estimators = 100,
                                    max_depth     = 8,
                                    random_state  = 123)

# FITTING the training data
ugbc_fit = ugbc.fit(x_train_1, y_train_1)


# PREDICTING on the response variable
ugbc_train_pred = ugbc_fit.predict(x_train_1)
ugbc_valid_pred = ugbc_fit.predict(x_train_2)

# # PREDICTING based on the testing set
# ugbc_pred = ugbc.predict(x_test)

# SCORING the results
ugbc_train_score = ugbc.score(x_train_1, y_train_1).round(4) # using R-square
ugbc_test_score  = ugbc.score(x_train_2, y_train_2).round(4)   # using R-square
ugbc_gap         = abs(ugbc_train_score - ugbc_test_score).round(4)

# SCORING the results (auc)
ugbc_train_auc = roc_auc_score(y_true  = y_train_1,
                                y_score = ugbc_train_pred).round(decimals = 4)

ugbc_valid_auc = roc_auc_score(y_true  = y_train_2,
                                y_score = ugbc_valid_pred).round(decimals = 4)

print (f"""
    Model Type       : {model_name}
    Training ACCURACY: {ugbc_train_score}
    Testing ACCURACY : {ugbc_test_score}
    Train-Test Gap   : {ugbc_gap}
    AUC Train        : {ugbc_train_auc}
    AUC Test         : {ugbc_valid_auc}
    """)


    Model Type       : Unpruned Gradient Boosting Classifier
    Training ACCURACY: 0.8245
    Testing ACCURACY : 0.8137
    Train-Test Gap   : 0.0108
    AUC Train        : 0.8244
    AUC Test         : 0.8136
    


In [14]:
# # Fitting it in the train_set
# x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
#             x_train,
#             y_train,
#             random_state = 123,
#             test_size    = 0.25,
#             stratify     = y_train)

# #Gradient Boosting
# # declaring a hyperparameter space
# criterion_range             = ['friedman_mse', 'squared_error', 'mse']
# loss_range                  = ['exponential', 'deviance']
# depth_range                 = range(1, 9, 1)
# leaf_range                  = range(1, 50, 1)
# min_samples_split_range      = range(1,20,1)

# # creating a hyperparameter grid
# param_grid = {'criterion'       : criterion_range,
#             'loss'              : loss_range,
#             'max_depth'         : depth_range,
#             'min_samples_leaf'  : leaf_range,
#             'min_samples_split' : min_samples_split_range}


# model_name = 'Pruned Gradient Boosting Classifier'

# # INSTANTIATING a classification tree object
# gradient_boosting = GradientBoostingClassifier(random_state = 219)

# # RandomizedSearchCV object
# gradient_boosting_cv = RandomizedSearchCV(estimator   = gradient_boosting,
#                                 param_distributions   = param_grid,
#                                 cv                    = 5,
#                                 n_iter                = 10,
#                                 random_state          = 123,
#                                 scoring               = 'roc_auc')

# # FITTING the training data
# gradient_boosting_fit = gradient_boosting_cv.fit(x_train_1, y_train_1)

# # PREDICTING on the response variable
# model_train_pred = gradient_boosting_fit.predict(x_train_1)
# model_valid_pred = gradient_boosting_fit.predict(x_train_2)

# # saving scoring data for future use
# gradient_boosting_train_score = gradient_boosting_fit.score(x_train_1, y_train_1).round(4) # accuracy

# gradient_boosting_test_score  = gradient_boosting_fit.score(x_train_2, y_train_2).round(4)   # accuracy

# gradient_boosting_model_gap   = abs(gradient_boosting_train_score - 
#                             gradient_boosting_test_score).round(4)

# # saving AUC
# gradient_boosting_train_auc_score   = roc_auc_score(y_true  = y_train_1,
#                                     y_score = model_train_pred).round(4) # auc
# gradient_boosting_test_auc_score   = roc_auc_score(y_true  = y_train_2,
#                                     y_score = model_valid_pred).round(4) # auc

# gradient_boosting_cv.best_params_



In [15]:
# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train,
            random_state = 123,
            test_size    = 0.25,
            stratify     = y_train)

#Gradient Boosting
# specifying a model name
model_name = 'Unpruned Gradient Boosting Classifier'

# INSTANTIATING the model object without hyperparameters
ugbc = GradientBoostingClassifier(loss          = 'exponential',
                                    min_samples_split= 12,
                                    min_samples_leaf= 44,
                                    criterion     = 'squared_error',
                                    max_depth     = 8,
                                    random_state  = 123)

# FITTING the training data
ugbc_fit = ugbc.fit(x_train_1, y_train_1)


# PREDICTING on the response variable
ugbc_train_pred = ugbc_fit.predict(x_train_1)
ugbc_valid_pred = ugbc_fit.predict(x_train_2)

# # PREDICTING based on the testing set
# ugbc_pred = ugbc.predict(x_test)

# SCORING the results
ugbc_train_score = ugbc.score(x_train_1, y_train_1).round(4) # using R-square
ugbc_test_score  = ugbc.score(x_train_2, y_train_2).round(4)   # using R-square
ugbc_gap         = abs(ugbc_train_score - ugbc_test_score).round(4)

# SCORING the results (auc)
ugbc_train_auc = roc_auc_score(y_true  = y_train_1,
                                y_score = ugbc_train_pred).round(decimals = 4)

ugbc_valid_auc = roc_auc_score(y_true  = y_train_2,
                                y_score = ugbc_valid_pred).round(decimals = 4)

print (f"""
    Model Type       : {model_name}
    Training ACCURACY: {ugbc_train_score}
    Testing ACCURACY : {ugbc_test_score}
    Train-Test Gap   : {ugbc_gap}
    AUC Train        : {ugbc_train_auc}
    AUC Test         : {ugbc_valid_auc}
    """)


    Model Type       : Unpruned Gradient Boosting Classifier
    Training ACCURACY: 0.8425
    Testing ACCURACY : 0.8036
    Train-Test Gap   : 0.0389
    AUC Train        : 0.8423
    AUC Test         : 0.8034
    


In [16]:
x_test = titanic_df[["CryoSleep_TRUE",
                      "CryoSleep_False",
                      "Wealthy_Person",
                      "With_RoomService",
                      "With_Spa",
                      "With_VRDeck",
                      "With_ShoppingMall",
                      "RoomService",
                      "Spa",
                      "VRDeck",
                      "Total_Lux_Spending",
                      "Earth_To_TRAPPIST",
                      "Deck_C",
                      "Side_P",
                      "Deck_E",
                      "With_Family",
                      "Mars",
                      "Teenagers",
                      "PSO",
                      "Earth_To_Cancrie",
                      'More_than_one_pax'
                      ]].copy()[ titanic_df['set'] == 'Testing' ]

# # setting x_test
# x_test  = titanic_df.drop(columns=["Transported_New", "Transported", "Name", "set"], axis=1).copy()[ titanic_df['set'] == 'Testing' ]


# PREDICTING on new data
model_pred = ugbc.predict(x_test)

# checking results
model_pred

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [17]:
# saving predictions with their respective Ids from the test set
predictions = pd.DataFrame(data = { 'PassengerId' : titanic_test['PassengerId'],
                                    'Transported' : model_pred.astype(bool)               } )

# checking the results
predictions.head(n = 5)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [18]:
# sending predictions to .csv file
predictions.to_csv(path_or_buf = 'submission_Team_1.csv',
                   index = False)

<br>